In [1]:
!pip install biopython

     |████████████████████████████████| 2.3MB 3.9MB/s 


In [2]:
!pip install transformers

     |████████████████████████████████| 2.3MB 3.8MB/s 
     |████████████████████████████████| 3.3MB 36.6MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 


Aqi vamos a hacer todo el rollo de cargar solo el txt que ya se secuencio de las cadenas del gen GRAP1

In [80]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

In [81]:
#inicializamos variables
RANDOM_SEED=42
MAX_LEN=200
BATCH_SIZE=16
DATASET_PATH='/content/drive/My Drive/ProyectInt/inEmbSegCancer.csv'
N_CLASES=2
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [82]:
#cargar dataset
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
df=pd.read_csv(DATASET_PATH)
df=df[0:10000]

Mounted at /content/drive


In [83]:
print(df.head())
print(df.shape)
print("\n".join(wrap(df["segmento"][147])))

                                            segmento    tipoVar
0  AUGGCAGAGGGCAAGGCAGGCGGCGCGGCCGGCCUCUUCGCCAAGC...  segm-norm
1  CCCAGGAGAAGGUAGGUGGCCGGGCCUCGCUGUCUGUCUGGCCCAC...  segm-norm
2  CCGGCCGGGCAGGAGGGGCGGGCAGGGUGCUCAGGGGCCCCGAAGG...  segm-norm
3  CCGCUCUCCCAGGGAAACGGGAGCGAGGAGUGGGCUUCCAUGAAAA...  segm-norm
4  GACCGCCACCCGAUCUUGGGCAAGGAUAAGGGAAGCUGGGGAGACA...  segm-norm
(148, 2)
AAUCAUGUUUACUUUCCAGGUAUCCACAAGUGAAAAUCCACAACUCUGA


In [84]:
df['label']=(df['tipoVar']=="protein-cancer").astype(int)
df.drop('tipoVar',axis=1,inplace=True)
df.head()

,segmento,label
0,AUGGCAGAGGGCAAGGCAGGCGGCGCGGCCGGCCUCUUCGCCAAGC...,0
1,CCCAGGAGAAGGUAGGUGGCCGGGCCUCGCUGUCUGUCUGGCCCAC...,0
2,CCGGCCGGGCAGGAGGGGCGGGCAGGGUGCUCAGGGGCCCCGAAGG...,0
3,CCGCUCUCCCAGGGAAACGGGAGCGAGGAGUGGGCUUCCAUGAAAA...,0
4,GACCGCCACCCGAUCUUGGGCAAGGAUAAGGGAAGCUGGGGAGACA...,0


In [85]:
#TOKENIZAR
#cargar modelo ya pre-entrenado
PRE_TRAINED_MODEL_NAME='bert-base-cased'
tokenizer=BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


In [87]:
#Ejemplo del los tokens
simple_example='NNNNNNNNCCUGUGAAUCAUCAUAAAGAUAUUGUGAAUAUUAAGAACAAUAUCUGCAACAAGUCCUU'
tokens=tokenizer.tokenize(simple_example)
token_ids=tokenizer.convert_tokens_to_ids(tokens)
print("Segmento:",simple_example)
print("Tokens:",tokens)
print("Tokens numericos", token_ids)

Segmento: NNNNNNNNCCUGUGAAUCAUCAUAAAGAUAUUGUGAAUAUUAAGAACAAUAUCUGCAACAAGUCCUU
Tokens: ['N', '##N', '##N', '##N', '##N', '##N', '##N', '##NC', '##CU', '##G', '##U', '##GA', '##A', '##UC', '##A', '##UC', '##A', '##U', '##AA', '##AG', '##A', '##U', '##A', '##U', '##U', '##G', '##U', '##GA', '##A', '##U', '##A', '##U', '##U', '##AA', '##GA', '##AC', '##AA', '##U', '##A', '##UC', '##U', '##GC', '##AA', '##CA', '##AG', '##UC', '##CU', '##U']
Tokens numericos [151, 2249, 2249, 2249, 2249, 2249, 2249, 15517, 23520, 2349, 2591, 10583, 1592, 21986, 1592, 21986, 1592, 2591, 11189, 22689, 1592, 2591, 1592, 2591, 2591, 2349, 2591, 10583, 1592, 2591, 1592, 2591, 2591, 11189, 10583, 8101, 11189, 2591, 1592, 21986, 2591, 13478, 11189, 11356, 22689, 21986, 23520, 2591]


In [90]:
#tokens especiales para bert
encoding = tokenizer.encode_plus(
    simple_example,
    max_length = 52,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    padding = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [91]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

['[CLS]', 'N', '##N', '##N', '##N', '##N', '##N', '##N', '##NC', '##CU', '##G', '##U', '##GA', '##A', '##UC', '##A', '##UC', '##A', '##U', '##AA', '##AG', '##A', '##U', '##A', '##U', '##U', '##G', '##U', '##GA', '##A', '##U', '##A', '##U', '##U', '##AA', '##GA', '##AC', '##AA', '##U', '##A', '##UC', '##U', '##GC', '##AA', '##CA', '##AG', '##UC', '##CU', '##U', '[SEP]']
tensor([  101,   151,  2249,  2249,  2249,  2249,  2249,  2249, 15517, 23520,
         2349,  2591, 10583,  1592, 21986,  1592, 21986,  1592,  2591, 11189,
        22689,  1592,  2591,  1592,  2591,  2591,  2349,  2591, 10583,  1592,
         2591,  1592,  2591,  2591, 11189, 10583,  8101, 11189,  2591,  1592,
        21986,  2591, 13478, 11189, 11356, 22689, 21986, 23520,  2591,   102])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1])


In [112]:
#Creacion del dataset

class segmentDataSet(Dataset):

  def __init__(self,segmento,labels,tokenizer,max_len):
    self.segmento = segmento
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.segmento)
    
  def __getitem__(self, item):
    segmento = str(self.segmento[item])
    label = self.labels[item]
    encoding = tokenizer.encode_plus(
        segmento,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        padding = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )
    

    return {
          'Segmento': segmento,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      }

In [113]:
#cargar los datos
def data_loader(df, tokenizer, max_len, batch_size):
  dataset = segmentDataSet(
      segmento = df.segmento.to_numpy(),
      labels = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )

  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 2)


In [114]:
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=RANDOM_SEED)
train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [121]:
#MOdelo BERT
class BERTadn(nn.Module):

  def __init__(self, n_classes):
    super(BERTadn, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, cls_output = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    drop_output = self.drop(cls_output)
    output = self.linear(drop_output)
    return output

In [141]:
model=BERTadn(N_CLASES)
model=model.to(device)
print(model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTadn(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [117]:
# ENTRENAMIENTO
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

In [133]:
# Iteración entrenamiento
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  batch=BATCH_SIZE
  for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [139]:
def classifySegment(segment_text):
  encoding_segment = tokenizer.encode_plus(
      segment_text,
      max_length = MAX_LEN,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      padding = True,
      return_attention_mask = True,
      return_tensors = 'pt'
      )
  
  input_ids = encoding_segment['input_ids'].to(device)
  attention_mask = encoding_segment['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1)
  print("\n".join(wrap(segment_text)))
  if prediction:
    print('Sentimiento predicho: Sin variacion Cancer')
  else:
    print('Sentimiento predicho: Cancer')


In [140]:
# Entrenamiento
for epoch in range(EPOCHS):
  print('Epoch {} de {}'.format(epoch+1, EPOCHS))
  print('------------------')
  train_acc, train_loss = train_model(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train))
  test_acc, test_loss = eval_model(model, test_data_loader, loss_fn, device, len(df_test))
  print('Entrenamiento: Loss: {}, accuracy: {}'.format(train_loss, train_acc))
  print('Validación: Loss: {}, accuracy: {}'.format(test_loss, test_acc))
  print('')

Epoch 1 de 5
------------------


RuntimeError: ignored

In [138]:
segment_text="UUUCAAAGAUUUUUACACUGCUCUCUACUGGUGUCAUAUUAGAUCACUAGGUGUAGAGUGCAGUACAUCU"
classifySegment(segment_text)

TypeError: ignored